In [1]:
!pip install transformers datasets seqeval

     |████████████████████████████████| 3.1 MB 4.3 MB/s 
     |████████████████████████████████| 290 kB 50.7 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 596 kB 40.5 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 3.3 MB 37.8 MB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 1.1 MB 39.2 MB/s 
     |████████████████████████████████| 132 kB 57.5 MB/s 
     |████████████████████████████████| 243 kB 54.0 MB/s 
     |████████████████████████████████| 192 kB 48.9 MB/s 
     |████████████████████████████████| 271 kB 51.6 MB/s 
     |████████████████████████████████| 160 kB 50.0 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=efce044ee7a916efb2d399e0a13530c26ce5eb7f7a45ca4a87fca5af2dee9be1
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72

# Fine-tuning language models for NER

## Load Dataset

In [ ]:
import datasets 
conll2003 = datasets.load_dataset("conll2003")

Downloading:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/650k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/163k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/40e7cb6bcc374f7c349c83acd1e9352a4f09474eb691f64f364ee62eb65d0ca6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

**Sample Data**

In [ ]:
conll2003["train"][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

In [ ]:
conll2003["train"][0:1]

{'chunk_tags': [[11, 21, 11, 12, 21, 22, 11, 12, 0]],
 'id': ['0'],
 'ner_tags': [[3, 0, 7, 0, 0, 0, 7, 0, 0]],
 'pos_tags': [[22, 42, 16, 21, 35, 37, 16, 21, 7]],
 'tokens': [['EU',
   'rejects',
   'German',
   'call',
   'to',
   'boycott',
   'British',
   'lamb',
   '.']]}

Notice the difference in the output if you access a single element vs. a slice of the data

**List of Tags**

In [ ]:
conll2003["train"].features["ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

## Tokenizer

In [ ]:
from transformers import BertTokenizerFast 
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased") 

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

The tokenizer is generally used to split a sentence into tokens and then map them to their token ids. However, as you can see, the input sentences have already been tokenized.

To tell the tokenizer that the sentences have already been tokenized, pass the `is_split_into_words=True` argument.

In [ ]:
tokenizer(["Oh","this","sentence","is","tokenized","and","splitted","by","spaces"], is_split_into_words=True)

{'input_ids': [101, 2821, 2023, 6251, 2003, 19204, 3550, 1998, 3975, 3064, 2011, 7258, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

If you don't, then each tokenized word will be treated as a sentence!

In [ ]:
tokenizer(["Oh","this","sentence","is","tokenized","and", "splitted","by","spaces"])

{'input_ids': [[101, 2821, 102], [101, 2023, 102], [101, 6251, 102], [101, 2003, 102], [101, 19204, 3550, 102], [101, 1998, 102], [101, 3975, 3064, 102], [101, 2011, 102], [101, 7258, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1, 1], [1, 1, 1]]}

## Alignment of Dataset

This function has a couple of tasks
- If it encounters a special token such as [CLS] or [SEP], give it an NER tag of `-100`. This is obtained by using the `word_ids` of the tokenized inputs, where these tokens get the `word_id = None`.
- If the token is a word that has not been splitted into sub-words, then assign it the NER tag mentioned in the original dataset
- If the token is a word that has been splitted into sub-words, then for the first token, assign it the original NER tag. For subsequent tokens (of the same word)
    - either assign them the same NER tag if `label_all_tokens=True`  
    - or assign the tag -100 if `label_all_tokens=False`.  \
  This is obtained by using the `word_ids` of the tokenized inputs, where splitted words get the same word_id.

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True): 
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True) 
    labels = [] 
    for i, label in enumerate(examples["ner_tags"]): 
        word_ids = tokenized_inputs.word_ids(batch_index=i) 
        previous_word_idx = None 
        label_ids = [] 
        for word_idx in word_ids: 
            if word_idx is None: 
                label_ids.append(-100) 
            elif word_idx != previous_word_idx: 
                 label_ids.append(label[word_idx]) 
            else: 
                 label_ids.append(label[word_idx] if label_all_tokens else -100) 
            previous_word_idx = word_idx 
        labels.append(label_ids) 
    tokenized_inputs["labels"] = labels 
    return tokenized_inputs 

In [ ]:
p = tokenize_and_align_labels(conll2003['train'][4:5], label_all_tokens=True) 
print(p) 

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 5, 0, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]]}


In [ ]:
q = tokenize_and_align_labels(conll2003['train'][4:5], label_all_tokens=False) 
print(q) 

{'input_ids': [[101, 2762, 1005, 1055, 4387, 2000, 1996, 2647, 2586, 1005, 1055, 15651, 2837, 14121, 1062, 9328, 5804, 2056, 2006, 9317, 10390, 2323, 4965, 8351, 4168, 4017, 2013, 3032, 2060, 2084, 3725, 2127, 1996, 4045, 6040, 2001, 24509, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 5, 0, -100, 0, 0, 0, 3, 4, 0, -100, 0, 0, 1, 2, -100, -100, 0, 0, 0, 0, 0, 0, 0, -100, -100, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, -100]]}


In [ ]:
for token, label_p, label_q in zip(tokenizer.convert_ids_to_tokens(p["input_ids"][0]),p["labels"][0], q["labels"][0]): 
    print(f"{token:_<30} {label_p:_<10} {label_q}") 

[CLS]_________________________ -100______ -100
germany_______________________ 5_________ 5
'_____________________________ 0_________ 0
s_____________________________ 0_________ -100
representative________________ 0_________ 0
to____________________________ 0_________ 0
the___________________________ 0_________ 0
european______________________ 3_________ 3
union_________________________ 4_________ 4
'_____________________________ 0_________ 0
s_____________________________ 0_________ -100
veterinary____________________ 0_________ 0
committee_____________________ 0_________ 0
werner________________________ 1_________ 1
z_____________________________ 2_________ 2
##wing________________________ 2_________ -100
##mann________________________ 2_________ -100
said__________________________ 0_________ 0
on____________________________ 0_________ 0
wednesday_____________________ 0_________ 0
consumers_____________________ 0_________ 0
should________________________ 0_________ 0
buy______________

## Tokenize the Dataset

In [ ]:
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/15 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

## Model

In [ ]:
from transformers import AutoModelForTokenClassification 
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

## Data Collator

In [ ]:
from transformers import DataCollatorForTokenClassification 
data_collator = DataCollatorForTokenClassification(tokenizer) 

## Evaluation on Dataset

seqeval is a good Python framework to evaluate sequence tagging algorithms and models

In [ ]:
!pip install seqeval

In [ ]:
metric = datasets.load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

**Metric on a Sample Dataset**

In [ ]:
example = conll2003['train'][0]

In [ ]:
label_list = conll2003["train"].features["ner_tags"].feature.names 
print(label_list)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [ ]:
labels = [label_list[i] for i in example["ner_tags"]]
print(labels)

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [ ]:
metric.compute(predictions=[labels], references=[labels]) 

{'MISC': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'ORG': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

**Compute Metrics for the Dataset**

In [ ]:
import numpy as np 
def compute_metrics(p): 
    predictions, labels = p 
    predictions = np.argmax(predictions, axis=2) 
    true_predictions = [ 
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100] 
        for prediction, label in zip(predictions, labels) 
    ] 
    true_labels = [ 
      [label_list[l] for (p, l) in zip(prediction, label) if l != -100] 
       for prediction, label in zip(predictions, labels) 
   ] 
    results = metric.compute(predictions=true_predictions, references=true_labels) 
    return { 
   "precision": results["overall_precision"], 
   "recall": results["overall_recall"], 
   "f1": results["overall_f1"], 
  "accuracy": results["overall_accuracy"], 
  } 

## Trainer

In [ ]:
from transformers import TrainingArguments, Trainer 
args = TrainingArguments( 
"test-ner",
evaluation_strategy = "epoch", 
learning_rate=2e-5, 
per_device_train_batch_size=16, 
per_device_eval_batch_size=16, 
num_train_epochs=3, 
weight_decay=0.01, 
) 

In [ ]:
trainer = Trainer( 
    model, 
    args, 
   train_dataset=tokenized_datasets["train"], 
   eval_dataset=tokenized_datasets["validation"], 
   data_collator=data_collator, 
   tokenizer=tokenizer, 
   compute_metrics=compute_metrics 
) 

In [ ]:
trainer.train() 

The following columns in the training set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, chunk_tags, tokens, ner_tags, pos_tags.
***** Running training *****
  Num examples = 14041
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2634


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.220300,0.063280,0.926174,0.929075,0.927622,0.983288
2,0.047000,0.060337,0.928824,0.941604,0.935170,0.984908
3,0.026200,0.056941,0.935117,0.946415,0.940732,0.986163


Saving model checkpoint to test-ner/checkpoint-500
Configuration saved in test-ner/checkpoint-500/config.json
Model weights saved in test-ner/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, chunk_tags, tokens, ner_tags, pos_tags.
***** Running Evaluation *****
  Num examples = 3250
  Batch size = 16
Saving model checkpoint to test-ner/checkpoint-1000
Configuration saved in test-ner/checkpoint-1000/config.json
Model weights saved in test-ner/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-ner/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-ner/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-ner/checkpoint-1500
Configuration saved in tes

TrainOutput(global_step=2634, training_loss=0.07799847069887293, metrics={'train_runtime': 941.6967, 'train_samples_per_second': 44.731, 'train_steps_per_second': 2.797, 'total_flos': 1019752160161410.0, 'train_loss': 0.07799847069887293, 'epoch': 3.0})

## Saving Model/Tokenizer

In [ ]:
model.save_pretrained("ner_model")

Configuration saved in ner_model/config.json
Model weights saved in ner_model/pytorch_model.bin


In [ ]:
tokenizer.save_pretrained("tokenizer")

tokenizer config file saved in tokenizer/tokenizer_config.json
Special tokens file saved in tokenizer/special_tokens_map.json


('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

## Using Model with Pipeline

In [ ]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i,label in enumerate(label_list)
}

In [ ]:
import json
config = json.load(open("ner_model/config.json"))
config["id2label"] = id2label
config["label2id"] = label2id
json.dump(config, open("ner_model/config.json","w"))

In [ ]:
mmodel = AutoModelForTokenClassification.from_pretrained("ner_model")

loading configuration file ner_model/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-MISC": "7",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-MISC": "8",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32

In [ ]:
from transformers import pipeline

In [ ]:
nlp = pipeline("ner", model=mmodel, tokenizer=tokenizer)
example = "I live in Istanbul"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-LOC', 'score': 0.99812406, 'index': 4, 'word': 'istanbul', 'start': 10, 'end': 18}]


In [ ]:
nlp = pipeline("ner", model=mmodel, tokenizer=tokenizer)
example = "John lives in Istanbul"

ner_results = nlp(example)
print(ner_results)

[{'entity': 'B-PER', 'score': 0.9957396, 'index': 1, 'word': 'john', 'start': 0, 'end': 4}, {'entity': 'B-LOC', 'score': 0.9980996, 'index': 4, 'word': 'istanbul', 'start': 14, 'end': 22}]


# Question answering using token classification

## SQUAD Dataset

### Version 1

In [2]:
from pprint import pprint 
from datasets import load_dataset 
squad = load_dataset("squad") 
for item in squad["train"][1].items(): 
    print(item[0]) 
    pprint(item[1]) 
    print("="*20) 

Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

id
'5733be284776f4190066117f'
title
'University_of_Notre_Dame'
context
('Architecturally, the school has a Catholic character. Atop the Main '
 "Building's gold dome is a golden statue of the Virgin Mary. Immediately in "
 'front of the Main Building and facing it, is a copper statue of Christ with '
 'arms upraised with the legend "Venite Ad Me Omnes". Next to the Main '
 'Building is the Basilica of the Sacred Heart. Immediately behind the '
 'basilica is the Grotto, a Marian place of prayer and reflection. It is a '
 'replica of the grotto at Lourdes, France where the Virgin Mary reputedly '
 'appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive '
 '(and in a direct line that connects through 3 statues and the Gold Dome), is '
 'a simple, modern stone statue of Mary.')
question
'What is in front of the Notre Dame Main Building?'
answers
{'answer_start': [188], 'text': ['a copper statue of Christ']}


### Version 2

In [3]:
from datasets import load_dataset 
squad = load_dataset("squad_v2") 
for item in squad["train"][1].items(): 
    print(item[0]) 
    pprint(item[1]) 
    print("="*20) 

Downloading:   0%|          | 0.00/1.87k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/9.55M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad_v2 downloaded and prepared to /root/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

id
'56be85543aeaaa14008c9065'
title
'Beyoncé'
context
('Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, '
 '1981) is an American singer, songwriter, record producer and actress. Born '
 'and raised in Houston, Texas, she performed in various singing and dancing '
 'competitions as a child, and rose to fame in the late 1990s as lead singer '
 "of R&B girl-group Destiny's Child. Managed by her father, Mathew Knowles, "
 "the group became one of the world's best-selling girl groups of all time. "
 "Their hiatus saw the release of Beyoncé's debut album, Dangerously in Love "
 '(2003), which established her as a solo artist worldwide, earned five Grammy '
 'Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" '
 'and "Baby Boy".')
question
'What areas did Beyonce compete in when she was growing up?'
answers
{'answer_start': [207], 'text': ['singing and dancing']}


In [4]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

## Tokenizer

In [5]:
from transformers import AutoTokenizer 
model_path = "distilbert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(model_path) 

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

## Parsing One Sample

In [6]:
max_length = 384 # maxlength after which the document will be split
doc_stride = 128 # length of tokens to be shared between the splitted segments
example = squad["train"][173] 
tokenized_example = tokenizer( 
                                example["question"], 
                                example["context"], 
                                max_length=max_length, 
                                truncation="only_second", 
                                return_overflowing_tokens=True, 
                                stride=doc_stride 
                                ) 

In [7]:
tokenized_example.keys()

dict_keys(['input_ids', 'attention_mask', 'overflow_to_sample_mapping'])

If we pass a long sentence, they will be split with each segment no longer than `max_length` (= 384 in our case) 

In [8]:
len(tokenized_example['input_ids'])

2

In [9]:
len(tokenized_example['input_ids'][0]),  len(tokenized_example['input_ids'][1])

(384, 192)

In [10]:
print(tokenized_example['input_ids'])

[[101, 20773, 2288, 2496, 1999, 2263, 2000, 3183, 1029, 102, 2006, 2258, 1018, 1010, 2263, 1010, 20773, 2496, 6108, 1062, 1012, 2016, 7271, 3936, 2037, 3510, 1999, 1037, 2678, 18318, 4270, 2012, 1996, 5962, 2283, 2005, 2014, 2353, 2996, 2201, 1010, 1045, 2572, 1012, 1012, 1012, 14673, 9205, 1010, 1999, 7128, 1005, 1055, 8412, 2252, 2006, 2255, 2570, 1010, 2263, 1012, 1045, 2572, 1012, 1012, 1012, 14673, 9205, 2001, 2207, 2006, 2281, 2324, 1010, 2263, 1999, 1996, 2142, 2163, 1012, 1996, 2201, 6246, 13999, 20773, 1005, 1055, 11477, 13059, 14673, 9205, 1010, 10141, 2076, 1996, 2437, 1997, 2014, 2494, 2309, 1000, 4689, 1999, 2293, 1000, 1010, 4855, 4466, 2475, 1010, 2199, 4809, 1999, 2049, 2034, 2733, 1010, 24469, 10234, 1996, 4908, 3263, 1010, 1998, 3228, 20773, 2014, 2353, 5486, 2193, 1011, 2028, 2201, 1999, 1996, 2149, 1012, 1996, 2201, 2956, 1996, 2193, 1011, 2028, 2299, 1000, 2309, 6456, 1006, 2404, 1037, 3614, 2006, 2009, 1007, 1000, 1998, 1996, 2327, 1011, 2274, 2774, 1000, 2065, 10

In [11]:
for input_ids in tokenized_example["input_ids"][:2]: 
    pprint(tokenizer.decode(input_ids)) 
    print("-"*50) 

('[CLS] beyonce got married in 2008 to whom? [SEP] on april 4, 2008, beyonce '
 'married jay z. she publicly revealed their marriage in a video montage at '
 'the listening party for her third studio album, i am... sasha fierce, in '
 "manhattan's sony club on october 22, 2008. i am... sasha fierce was released "
 'on november 18, 2008 in the united states. the album formally introduces '
 "beyonce's alter ego sasha fierce, conceived during the making of her 2003 "
 'single " crazy in love ", selling 482, 000 copies in its first week, '
 'debuting atop the billboard 200, and giving beyonce her third consecutive '
 'number - one album in the us. the album featured the number - one song " '
 'single ladies ( put a ring on it ) " and the top - five songs " if i were a '
 'boy " and " halo ". achieving the accomplishment of becoming her longest - '
 'running hot 100 single in her career, " halo "\'s success in the us helped '
 'beyonce attain more top - ten singles on the list than any oth

In [12]:
len([2309, 6456, 1000, 2038, 2042, 11968, 7716, 6340, 1998, 10047, 15198, 2105, 1996, 2088, 1010, 27957, 1996, 1000, 2034, 2350, 3153, 13675, 10936, 2063, 1000, 1997, 1996, 4274, 2287, 2429, 2000, 1996, 4361, 2732, 1012, 1996, 2678, 2038, 2180, 2195, 2982, 1010, 2164, 2190, 2678, 2012, 1996, 2268, 8692, 2885, 2189, 2982, 1010, 1996, 2268, 4104, 11240, 2080, 2982, 1010, 1998, 1996, 2268, 6655, 2982, 1012, 2012, 1996, 2268, 8692, 2678, 2189, 2982, 1010, 1996, 2678, 2001, 4222, 2005, 3157, 2982, 1010, 4821, 3045, 2093, 2164, 2678, 1997, 1996, 2095, 1012, 2049, 4945, 2000, 2663, 1996, 2190, 2931, 2678, 4696, 1010, 2029, 2253, 2000, 2137, 2406, 3769, 3220, 4202, 9170, 1005, 1055, 1000, 2017, 7141, 2007, 2033, 1000, 1010, 2419, 2000, 29270, 2225, 22602, 1996, 5103, 1998, 20773])

128

This is `doc_stride`; the tokens shared between the splitted segments

## Processing One Sample

Few things we need to do.
- the `end_span` is not specified. We need to add it to the dataset'
- the start and end spans indicate the span of characters not words/tokens. Our model is designed to predict the tokens!
- samples with longer context are actually converted to multiple samples (i.e. each offset is considered to be a sample) Therefore, our tokenized and processed dataset will be longer than the original dataset!

In [13]:
example = squad["train"][173:174] 
example

{'answers': [{'answer_start': [34], 'text': ['Jay Z']}],
 'context': ['On April 4, 2008, Beyoncé married Jay Z. She publicly revealed their marriage in a video montage at the listening party for her third studio album, I Am... Sasha Fierce, in Manhattan\'s Sony Club on October 22, 2008. I Am... Sasha Fierce was released on November 18, 2008 in the United States. The album formally introduces Beyoncé\'s alter ego Sasha Fierce, conceived during the making of her 2003 single "Crazy in Love", selling 482,000 copies in its first week, debuting atop the Billboard 200, and giving Beyoncé her third consecutive number-one album in the US. The album featured the number-one song "Single Ladies (Put a Ring on It)" and the top-five songs "If I Were a Boy" and "Halo". Achieving the accomplishment of becoming her longest-running Hot 100 single in her career, "Halo"\'s success in the US helped Beyoncé attain more top-ten singles on the list than any other woman during the 2000s. It also included the s

In [14]:
example['answers'][0]['answer_start'][0]

34

In [15]:
example['context'][0][34:39]

'Jay Z'

In [16]:
max_length = 384 # maxlength after which the document will be split
doc_stride = 128 # length of tokens to be shared between the splitted segments
tokenized_examples = tokenizer( 
        example["question"], 
        example["context"], 
        max_length=max_length, 
        truncation="only_second", 
        padding="max_length", 
        return_offsets_mapping=True,
        return_overflowing_tokens=True, 
        stride=doc_stride
        ) 

In [18]:
print(tokenized_examples)

{'input_ids': [[101, 20773, 2288, 2496, 1999, 2263, 2000, 3183, 1029, 102, 2006, 2258, 1018, 1010, 2263, 1010, 20773, 2496, 6108, 1062, 1012, 2016, 7271, 3936, 2037, 3510, 1999, 1037, 2678, 18318, 4270, 2012, 1996, 5962, 2283, 2005, 2014, 2353, 2996, 2201, 1010, 1045, 2572, 1012, 1012, 1012, 14673, 9205, 1010, 1999, 7128, 1005, 1055, 8412, 2252, 2006, 2255, 2570, 1010, 2263, 1012, 1045, 2572, 1012, 1012, 1012, 14673, 9205, 2001, 2207, 2006, 2281, 2324, 1010, 2263, 1999, 1996, 2142, 2163, 1012, 1996, 2201, 6246, 13999, 20773, 1005, 1055, 11477, 13059, 14673, 9205, 1010, 10141, 2076, 1996, 2437, 1997, 2014, 2494, 2309, 1000, 4689, 1999, 2293, 1000, 1010, 4855, 4466, 2475, 1010, 2199, 4809, 1999, 2049, 2034, 2733, 1010, 24469, 10234, 1996, 4908, 3263, 1010, 1998, 3228, 20773, 2014, 2353, 5486, 2193, 1011, 2028, 2201, 1999, 1996, 2149, 1012, 1996, 2201, 2956, 1996, 2193, 1011, 2028, 2299, 1000, 2309, 6456, 1006, 2404, 1037, 3614, 2006, 2009, 1007, 1000, 1998, 1996, 2327, 1011, 2274, 2774, 

In [19]:
print(tokenizer.convert_ids_to_tokens(tokenized_examples["input_ids"][0]))

['[CLS]', 'beyonce', 'got', 'married', 'in', '2008', 'to', 'whom', '?', '[SEP]', 'on', 'april', '4', ',', '2008', ',', 'beyonce', 'married', 'jay', 'z', '.', 'she', 'publicly', 'revealed', 'their', 'marriage', 'in', 'a', 'video', 'mont', '##age', 'at', 'the', 'listening', 'party', 'for', 'her', 'third', 'studio', 'album', ',', 'i', 'am', '.', '.', '.', 'sasha', 'fierce', ',', 'in', 'manhattan', "'", 's', 'sony', 'club', 'on', 'october', '22', ',', '2008', '.', 'i', 'am', '.', '.', '.', 'sasha', 'fierce', 'was', 'released', 'on', 'november', '18', ',', '2008', 'in', 'the', 'united', 'states', '.', 'the', 'album', 'formally', 'introduces', 'beyonce', "'", 's', 'alter', 'ego', 'sasha', 'fierce', ',', 'conceived', 'during', 'the', 'making', 'of', 'her', '2003', 'single', '"', 'crazy', 'in', 'love', '"', ',', 'selling', '48', '##2', ',', '000', 'copies', 'in', 'its', 'first', 'week', ',', 'debuting', 'atop', 'the', 'billboard', '200', ',', 'and', 'giving', 'beyonce', 'her', 'third', 'consec

In [20]:
tokenizer.convert_ids_to_tokens(tokenized_examples["input_ids"][0])[18:20]

['jay', 'z']

Therefore the tokens start/end position is 18/19

**overflow_to_sample_mapping**: This tells us how the context has been split. In the following example, the first context (numbered 0) has been split into two segments (hence both have the same value - 0)

In [21]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping") 
sample_mapping

[0, 0]

**offset_mapping**: It is a list of tuples where each element is a tuple of the start and end index of the tokens. 

In [22]:
offset_mapping = tokenized_examples.pop("offset_mapping") 
offset_mapping

[[(0, 0),
  (0, 7),
  (8, 11),
  (12, 19),
  (20, 22),
  (23, 27),
  (28, 30),
  (31, 35),
  (35, 36),
  (0, 0),
  (0, 2),
  (3, 8),
  (9, 10),
  (10, 11),
  (12, 16),
  (16, 17),
  (18, 25),
  (26, 33),
  (34, 37),
  (38, 39),
  (39, 40),
  (41, 44),
  (45, 53),
  (54, 62),
  (63, 68),
  (69, 77),
  (78, 80),
  (81, 82),
  (83, 88),
  (89, 93),
  (93, 96),
  (97, 99),
  (100, 103),
  (104, 113),
  (114, 119),
  (120, 123),
  (124, 127),
  (128, 133),
  (134, 140),
  (141, 146),
  (146, 147),
  (148, 149),
  (150, 152),
  (152, 153),
  (153, 154),
  (154, 155),
  (156, 161),
  (162, 168),
  (168, 169),
  (170, 172),
  (173, 182),
  (182, 183),
  (183, 184),
  (185, 189),
  (190, 194),
  (195, 197),
  (198, 205),
  (206, 208),
  (208, 209),
  (210, 214),
  (214, 215),
  (216, 217),
  (218, 220),
  (220, 221),
  (221, 222),
  (222, 223),
  (224, 229),
  (230, 236),
  (237, 240),
  (241, 249),
  (250, 252),
  (253, 261),
  (262, 264),
  (264, 265),
  (266, 270),
  (271, 273),
  (274, 277)

Our dataset should have the elements `start_positions` and `end_positions`. Although the former is present in the dataset, but we have to extract it from the `answers` element.

In [23]:
tokenized_examples["start_positions"] = [] 
tokenized_examples["end_positions"] = [] 

Let's go over the first sample

We will iterate over the offsets (i.e. long sequences that have been splitted)

**First Offset**

In [24]:
# select first offset with question and first part of context
offset_num = 0

In [25]:
# get the start/end of all tokens
offsets = offset_mapping[offset_num]
print(offsets)

[(0, 0), (0, 7), (8, 11), (12, 19), (20, 22), (23, 27), (28, 30), (31, 35), (35, 36), (0, 0), (0, 2), (3, 8), (9, 10), (10, 11), (12, 16), (16, 17), (18, 25), (26, 33), (34, 37), (38, 39), (39, 40), (41, 44), (45, 53), (54, 62), (63, 68), (69, 77), (78, 80), (81, 82), (83, 88), (89, 93), (93, 96), (97, 99), (100, 103), (104, 113), (114, 119), (120, 123), (124, 127), (128, 133), (134, 140), (141, 146), (146, 147), (148, 149), (150, 152), (152, 153), (153, 154), (154, 155), (156, 161), (162, 168), (168, 169), (170, 172), (173, 182), (182, 183), (183, 184), (185, 189), (190, 194), (195, 197), (198, 205), (206, 208), (208, 209), (210, 214), (214, 215), (216, 217), (218, 220), (220, 221), (221, 222), (222, 223), (224, 229), (230, 236), (237, 240), (241, 249), (250, 252), (253, 261), (262, 264), (264, 265), (266, 270), (271, 273), (274, 277), (278, 284), (285, 291), (291, 292), (293, 296), (297, 302), (303, 311), (312, 322), (323, 330), (330, 331), (331, 332), (333, 338), (339, 342), (343, 3

In [26]:
# disregard the other offsets for a while
input_ids = tokenized_examples["input_ids"][offset_num]
print(input_ids)

[101, 20773, 2288, 2496, 1999, 2263, 2000, 3183, 1029, 102, 2006, 2258, 1018, 1010, 2263, 1010, 20773, 2496, 6108, 1062, 1012, 2016, 7271, 3936, 2037, 3510, 1999, 1037, 2678, 18318, 4270, 2012, 1996, 5962, 2283, 2005, 2014, 2353, 2996, 2201, 1010, 1045, 2572, 1012, 1012, 1012, 14673, 9205, 1010, 1999, 7128, 1005, 1055, 8412, 2252, 2006, 2255, 2570, 1010, 2263, 1012, 1045, 2572, 1012, 1012, 1012, 14673, 9205, 2001, 2207, 2006, 2281, 2324, 1010, 2263, 1999, 1996, 2142, 2163, 1012, 1996, 2201, 6246, 13999, 20773, 1005, 1055, 11477, 13059, 14673, 9205, 1010, 10141, 2076, 1996, 2437, 1997, 2014, 2494, 2309, 1000, 4689, 1999, 2293, 1000, 1010, 4855, 4466, 2475, 1010, 2199, 4809, 1999, 2049, 2034, 2733, 1010, 24469, 10234, 1996, 4908, 3263, 1010, 1998, 3228, 20773, 2014, 2353, 5486, 2193, 1011, 2028, 2201, 1999, 1996, 2149, 1012, 1996, 2201, 2956, 1996, 2193, 1011, 2028, 2299, 1000, 2309, 6456, 1006, 2404, 1037, 3614, 2006, 2009, 1007, 1000, 1998, 1996, 2327, 1011, 2274, 2774, 1000, 2065, 104

In [27]:
cls_index = input_ids.index(tokenizer.cls_token_id)
print(cls_index)

0


In [28]:
# 0 is first sequence (question), 1 is the second sequence (context), None means special tokens
sequence_ids = tokenized_examples.sequence_ids(offset_num) 
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [29]:
# One example can give several spans, this is the index of the example containing this span of text.
sample_index = sample_mapping[offset_num]
print(sample_index)
answers = example["answers"][sample_index]
print(answers)

0
{'text': ['Jay Z'], 'answer_start': [34]}


In [30]:
# If no answers are given, set the cls_index as answer.
if len(answers["answer_start"]) == 0:
    tokenized_examples["start_positions"].append(cls_index)
    tokenized_examples["end_positions"].append(cls_index)
    
print(tokenized_examples["start_positions"], tokenized_examples["end_positions"])

[] []


However, our example has an answer, so we will parse the other part

In [31]:
# Start/end character index of the answer in the text.
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])
print(start_char, end_char)

34 39


However, recall that this is for the characters, not tokens! 

In [32]:
# Start token index of the current span in the text.
# Acheived by finding the first occurence of 1 in the sequence_ids
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1
print(token_start_index)

10


In [33]:
# End token index of the current span in the text.
# Acheived by finding the last occurence of 1 in the sequence_ids
token_end_index = len(input_ids) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1
print(token_end_index)

382


In [34]:
# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    tokenized_examples["start_positions"].append(cls_index)
    tokenized_examples["end_positions"].append(cls_index)
print(tokenized_examples["start_positions"], tokenized_examples["end_positions"])

[] []


Since it is inside this span, therefore it is empty

In [35]:
# Detect if the answer is in this span 
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    tokenized_examples["start_positions"].append(token_start_index - 1)
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    tokenized_examples["end_positions"].append(token_end_index + 1)
print(tokenized_examples["start_positions"], tokenized_examples["end_positions"])

[18] [19]


This completes the first offset. Now we will do the same steps for the second offset.

**Second Offset**

In [36]:
# select first offset with question and first part of context
offset_num = 1

# get the start/end of all tokens
offsets = offset_mapping[offset_num]
print(offsets)

# disregard the other offsets for a while
input_ids = tokenized_examples["input_ids"][offset_num]
print(input_ids)

cls_index = input_ids.index(tokenizer.cls_token_id)
print(cls_index)

# 0 is first sequence (question), 1 is the second sequence (context), None means special tokens
sequence_ids = tokenized_examples.sequence_ids(offset_num) 
print(sequence_ids)

# One example can give several spans, this is the index of the example containing this span of text.
sample_index = sample_mapping[offset_num]
print(sample_index)
answers = example["answers"][sample_index]
print(answers)

# If no answers are given, set the cls_index as answer.
if len(answers["answer_start"]) == 0:
    tokenized_examples["start_positions"].append(cls_index)
    tokenized_examples["end_positions"].append(cls_index)
    
print(tokenized_examples["start_positions"], tokenized_examples["end_positions"])

# Start/end character index of the answer in the text.
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])
print(start_char, end_char)

# Start token index of the current span in the text.
# Acheived by finding the first occurence of 1 in the sequence_ids
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1
print(token_start_index)

# End token index of the current span in the text.
# Acheived by finding the last occurence of 1 in the sequence_ids
token_end_index = len(input_ids) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1
print(token_end_index)

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    tokenized_examples["start_positions"].append(cls_index)
    tokenized_examples["end_positions"].append(cls_index)

# Detect if the answer is in this span 
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    tokenized_examples["start_positions"].append(token_start_index - 1)
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    tokenized_examples["end_positions"].append(token_end_index + 1)
print(tokenized_examples["start_positions"], tokenized_examples["end_positions"])



[(0, 0), (0, 7), (8, 11), (12, 19), (20, 22), (23, 27), (28, 30), (31, 35), (35, 36), (0, 0), (1041, 1047), (1048, 1054), (1054, 1055), (1056, 1059), (1060, 1064), (1065, 1068), (1068, 1070), (1070, 1073), (1074, 1077), (1078, 1080), (1080, 1086), (1087, 1093), (1094, 1097), (1098, 1103), (1103, 1104), (1105, 1113), (1114, 1117), (1118, 1119), (1119, 1124), (1125, 1130), (1131, 1136), (1137, 1139), (1139, 1141), (1141, 1142), (1142, 1143), (1144, 1146), (1147, 1150), (1151, 1159), (1160, 1163), (1164, 1173), (1174, 1176), (1177, 1180), (1181, 1188), (1189, 1193), (1193, 1194), (1195, 1198), (1199, 1204), (1205, 1208), (1209, 1212), (1213, 1220), (1221, 1227), (1227, 1228), (1229, 1238), (1239, 1243), (1244, 1249), (1250, 1252), (1253, 1256), (1257, 1261), (1262, 1265), (1266, 1272), (1273, 1278), (1279, 1285), (1285, 1286), (1287, 1290), (1291, 1295), (1296, 1304), (1305, 1308), (1308, 1309), (1310, 1316), (1316, 1317), (1318, 1321), (1322, 1325), (1326, 1330), (1331, 1334), (1335, 134

Our sample was turned into two samples with the second one having 0 as the  start and end positions since our answer was present in the first offset.

## Processing Dataset

The function below performs some tasks.
- Works for both left-to-right and right-to-left languages. (defined using `pad_on_right` argument)
- performs the above steps mentioned for a single sample

In [37]:
def prepare_train_features(examples, pad_on_right=True): 
    tokenized_examples = tokenizer( 
        examples["question" if pad_on_right else "context"], 
        examples["context" if pad_on_right else "question"], 
        truncation="only_second" if pad_on_right else "only_first", 
        max_length=max_length, 
        stride=doc_stride, 
        return_overflowing_tokens=True, 
        return_offsets_mapping=True, 
        padding="max_length", 
    ) 
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping") 
    offset_mapping = tokenized_examples.pop("offset_mapping") 
    tokenized_examples["start_positions"] = [] 
    tokenized_examples["end_positions"] = [] 
    for i, offsets in enumerate(offset_mapping): 
        input_ids = tokenized_examples["input_ids"][i] 
        cls_index = input_ids.index(tokenizer.cls_token_id) 
        sequence_ids = tokenized_examples.sequence_ids(i) 
        sample_index = sample_mapping[i] 
        answers = examples["answers"][sample_index] 
        if len(answers["answer_start"]) == 0: 
            tokenized_examples["start_positions"].append(cls_index) 
            tokenized_examples["end_positions"].append(cls_index) 
        else: 
            start_char = answers["answer_start"][0] 
            end_char = start_char + len(answers["text"][0]) 
            token_start_index = 0 
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0): 
                token_start_index += 1 
            token_end_index = len(input_ids) - 1 
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0): 
                token_end_index -= 1 
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char): 
                tokenized_examples["start_positions"].append(cls_index) 
                tokenized_examples["end_positions"].append(cls_index) 
            else: 
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char: 
                    token_start_index += 1 
                tokenized_examples["start_positions"].append(token_start_index - 1) 
                while offsets[token_end_index][1] >= end_char: 
                    token_end_index -= 1 
                tokenized_examples["end_positions"].append(token_end_index + 1) 
    return tokenized_examples 

In [38]:
tokenized_datasets = squad.map(prepare_train_features, batched=True, remove_columns=squad["train"].column_names)

  0%|          | 0/131 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [40]:
print(tokenized_datasets['train'][173])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [42]:
print(tokenized_datasets['train'][174])

{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

**Change of Dataset Size because of Offsets/Overflow/Longer Sequence**

In [43]:
squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

In [44]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions'],
        num_rows: 131754
    })
    validation: Dataset({
        features: ['attention_mask', 'end_positions', 'input_ids', 'start_positions'],
        num_rows: 12134
    })
})

## Model

In [45]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer 
model_path = "distilbert-base-uncased" 
model = AutoModelForQuestionAnswering.from_pretrained(model_path) 

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

## Data Collator

In [46]:
from transformers import default_data_collator 
data_collator = default_data_collator 

## Trainer

In [50]:
args = TrainingArguments( 
                    f"test-squad", 
                    evaluation_strategy = "epoch", 
                    learning_rate=2e-5, 
                    per_device_train_batch_size=32, 
                    per_device_eval_batch_size=32, 
                    num_train_epochs=1, 
                    weight_decay=0.01, 
                    )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [51]:
trainer = Trainer( 
                model, 
                args, 
                train_dataset=tokenized_datasets["train"], 
                eval_dataset=tokenized_datasets["validation"], 
                data_collator=data_collator, 
                tokenizer=tokenizer, 
                ) 

In [52]:
trainer.train() 

***** Running training *****
  Num examples = 131754
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4118


Epoch,Training Loss,Validation Loss
1,1.367400,1.289845


Saving model checkpoint to test-squad/checkpoint-500
Configuration saved in test-squad/checkpoint-500/config.json
Model weights saved in test-squad/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-500/special_tokens_map.json
Saving model checkpoint to test-squad/checkpoint-1000
Configuration saved in test-squad/checkpoint-1000/config.json
Model weights saved in test-squad/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-1000/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to test-squad/checkpoint-1500
Configuration saved in test-squad/checkpoint-1500/config.json
Model weights saved in test-squad/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in test-squad/checkpoint-1500/tokenizer_config.json
Special tokens file saved in test-squad/checkpoint-15

TrainOutput(global_step=4118, training_loss=1.6394092509791711, metrics={'train_runtime': 9311.0103, 'train_samples_per_second': 14.15, 'train_steps_per_second': 0.442, 'total_flos': 1.2910551406212096e+16, 'train_loss': 1.6394092509791711, 'epoch': 1.0})

## Saving Model

In [53]:
trainer.save_model("distillBERT_SQUAD")

Saving model checkpoint to distillBERT_SQUAD
Configuration saved in distillBERT_SQUAD/config.json
Model weights saved in distillBERT_SQUAD/pytorch_model.bin
tokenizer config file saved in distillBERT_SQUAD/tokenizer_config.json
Special tokens file saved in distillBERT_SQUAD/special_tokens_map.json


## Loading Model for Use in Pipeline

In [54]:
from transformers import pipeline 
qa_model = pipeline('question-answering', model='distilbert-base-cased-distilled-squad', tokenizer='distilbert-base-cased') 

https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpa6refxnz


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
creating metadata file for /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
loading configuration file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/81e8dfe090123eff18dd06533ead3ae407b82e30834c50c7c82c2305ce3ace12.ca0305b1f128274fa0c6e4859d1c1477d0e34a20be25da95eea888b30ece9cf3
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max

Downloading:   0%|          | 0.00/249M [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/pytorch_model.bin in cache at /root/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
creating metadata file for /root/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
loading weights file https://huggingface.co/distilbert-base-cased-distilled-squad/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/02f483764e26e1acfddfcd6e4879785f2908b2806a962d01b888bfe2b988075b.bd96a9432b167ab5e2b086cf0b688ca3c43c027091377eddfcfd39bdde851c35
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-cased-distilled-squad.
If yo

...

storing https://huggingface.co/distilbert-base-cased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
creating metadata file for /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
loading configuration file https://huggingface.co/distilbert-base-cased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ebe1ea24d11aa664488b8de5b21e33989008ca78f207d4e30ec6350b693f073f.302bfd1b5e031cc1b17796e0b6e5b242ba2045d31d00f97589e12b458ebff27a
Model config DistilBertConfig {
  "activation": "gelu",
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/ba377304984dc63e3ede0e23a938bbbf04d5c3835b66d5bb48343aecca188429.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
creating metadata file for /root/.cache/huggingface/transformers/ba377304984dc63e3ede0e23a938bbbf04d5c3835b66d5bb48343aecca188429.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp0hdgx3n5


Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/acb5c2138c1f8c84f074b86dafce3631667fccd6efcb1a7ea1320cf75c386a36.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
creating metadata file for /root/.cache/huggingface/transformers/acb5c2138c1f8c84f074b86dafce3631667fccd6efcb1a7ea1320cf75c386a36.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
loading file https://huggingface.co/distilbert-base-cased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/ba377304984dc63e3ede0e23a938bbbf04d5c3835b66d5bb48343aecca188429.437aa611e89f6fc6675a049d2b5545390adbc617e7d655286421c191d2be2791
loading file https://huggingface.co/distilbert-base-cased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/acb5c2138c1f8c84f074b86dafce3631667fccd6efcb1a7ea1320cf75c386a36.3dab63143af66769bbb35e3811f75f7e16b2320e12b7935e216bd6159ce6d9a6
loading file 

In [55]:
from transformers import pipeline 
qa_model = pipeline('question-answering', model='distillBERT_SQUAD', tokenizer='distilbert-base-cased') 

loading configuration file distillBERT_SQUAD/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.12.5",
  "vocab_size": 30522
}

loading configuration file distillBERT_SQUAD/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_em

In [56]:
question = squad["validation"][0]["question"] 
context = squad["validation"][0]["context"] 
print("Question:") 
print(question) 
print("Context:") 
print(context) 

Question:
In what country is Normandy located?
Context:
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.


In [57]:
qa_model(question=question, context=context) 

{'answer': 'name to', 'end': 136, 'score': 0.0005753916339017451, 'start': 129}